# Import des librairies

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=ffc6afc41d6520c0d260b107315ae0de292e71fe135ed0692b3769979b9186ac
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
import pandas as pnd
from langdetect import detect
from google.colab import files

In [ ]:
depression_data = pnd.read_csv('depression_with_vader.csv',delimiter=',',engine='python',on_bad_lines='skip',encoding="utf-8",header=[0],index_col=[0])
depression_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45996 entries, 0 to 45992
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             45994 non-null  object 
 1   tweet                  45994 non-null  object 
 2   language               45993 non-null  object 
 3   clean_tweet            45990 non-null  object 
 4   vader_score            45991 non-null  float64
 5   vader_sentiment_label  45991 non-null  float64
dtypes: float64(2), object(4)
memory usage: 2.5+ MB


# Nettoyage des données

On récupère les lignes dont le label est à 0

In [ ]:
depression_data  = depression_data.loc[depression_data ['vader_sentiment_label'] == 0]
depression_data = depression_data.sample(30000)

In [ ]:
depression_data.tail(5)

,Unnamed: 0,tweet,language,clean_tweet,vader_score,vader_sentiment_label
31539,31539,Are you afraid of loneliness? — It's better to...,en,Are afraid loneliness? — It's better lonely fa...,-0.4137,0.0
44580,44580,CAN SOMEONE SAVE ME FROM MY DEPRESSION TOMORRO...,en,CAN SOMEONE SAVE ME FROM MY DEPRESSION TOMORRO...,-0.1280,0.0
29875,29875,I already feel that tomorrow I’m gonna wake up...,en,I already feel tomorrow I’m gonna wake hella d...,-0.5563,0.0
42814,42814,pic.twitter.com/aoxZgbjTJV,en,pic.twitter.com/aoxZgbjTJV,0.0000,0.0
6054,6054,This depression episode is really fucked up,en,This depression episode really fucked,-0.8553,0.0


Supppression des colonnes inutiles

In [ ]:
depression_data = depression_data.drop(['Unnamed: 0', 'tweet', 'language', 'vader_score'], axis=1)

In [ ]:
depression_data.rename(columns={'clean_tweet': 'tweet', 'vader_sentiment_label':'label'}, inplace=True)
depression_data.sample(100)

,tweet,label
8785,It's old play idea must reassure moneyed men i...,0.0
21355,Was full expecting put 2lbs week went lost pou...,0.0
38215,I'm embarrassed admit I've fallen,0.0
21632,I think,0.0
42900,I love protube stuff new cups hurt hand index ...,0.0
...,...,...
32525,Are afraid loneliness? — yes i'm lonely like 2...,0.0
5,react severe depression,0.0
11976,That bullshit. Trans suicide rates vastly infl...,0.0
7938,What It Feels Like Have 'Chronic Depression Fa...,0.0


In [ ]:
random_data = pnd.read_csv('sentiment.csv',delimiter=',',engine='python',on_bad_lines='skip',encoding="utf-8",header=[0],index_col=[0])
random_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93684 entries, 1 to 93703
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Sentiment        93684 non-null  int64 
 1   SentimentSource  93684 non-null  object
 2   SentimentText    93684 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [ ]:
random_data = random_data.drop(['SentimentSource'], axis=1)

Nettoyage des données, en ignorant toutes les publications des mots clés

In [ ]:
random_data.rename(columns={'SentimentText': 'tweet', 'Sentiment':'label'}, inplace=True)
random_data = random_data[~random_data['tweet'].str.contains('hopeless|lonely|depressive|depressed|suicide')]
random_data.sample(100)

,label,tweet
ItemID,,
7917,0,#rox now dwn 7-2 sumo wresting. http://twitpi...
87735,0,@churunga: I still have your gifts to send. I ...
4661,1,#cravesushi Looking for good music and beautif...
82251,0,@chelsea328. well my day is ruined
34826,1,@AkeenAnthony im nasty.... thats like day 4 th...
...,...,...
82471,1,@chemicalbullets awww. You're so sweet
85371,1,@CHRISDJMOYLES Dare you to play the Beastie Bo...
77889,1,@Cafechan Do iiiiiit. I would!


In [ ]:
random_data  = random_data.loc[random_data ['label'] == 1]
random_data.head(10)

,label,tweet
ItemID,,
3,1,omg its already 7:30 :O
7,1,Juuuuuuuuuuuuuuuuussssst Chillin!!
9,1,handed in my uniform today . i miss you ...
10,1,hmmmm.... i wonder how she my number @-)
12,1,thanks to all the haters up in my face a...
18,1,Feeling strangely fine. Now I'm gonna go l...
23,1,You're the only one who can see this cause...
29,1,"goodbye exams, HELLO ALCOHOL TONIGHT"
39,1,uploading pictures on friendster


In [ ]:
random_data = random_data.sample(30000)

Suppression des publications non anglaises

In [ ]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return ''

In [ ]:
random_data ['language'] = random_data ['tweet'].apply(detect_language)

random_data.head()

filtered_lines = random_data [random_data ['language'] == 'en']
filtered_lines = filtered_lines.reset_index(drop=True)

In [ ]:
depression_data['label'] = depression_data['label'].fillna(1)
random_data['label'] = random_data['label'].fillna(0)
random_data = random_data.drop(['language'], axis=1)

Dans un premier temps on fusionne nos 2 jeux de données. Dans un second temps, on inverse les valeurs des labels, pour avoir le résultat suivant 🇰


*   0 = publications à caractères non dépresifs
*   1 = publications à caractères dépressifs



In [ ]:
random_data.info()

final_df = pnd.concat([depression_data, random_data], ignore_index=True)

final_df.sample(100)
final_df['label'] = final_df['label'].apply(lambda x: 1 if x == 0 else 0)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 6729 to 36312
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   30000 non-null  int64 
 1   tweet   30000 non-null  object
dtypes: int64(1), object(1)
memory usage: 703.1+ KB


Nous avons un jeu de données avec autant d'étiquettes à 0 et 1

In [ ]:
nbr_label = final_df['label'].value_counts()
print(nbr_label)

1    30000
0    30000
Name: label, dtype: int64


In [ ]:
final_df.sample(100)

,tweet,label
54143,@Beeachh i love ginos,0
49097,"@alexderossi http://twitpic.com/4gvuo - grabe,...",0
33939,@avantgarden My purple harem... what would I d...,0
5209,"Taoris: Somebody I used know, Ocean Drive Litt...",1
10474,Me: ugh I feel shitty sad I literally get noth...,1
...,...,...
52822,@bagelsss yes'm. have been working on a reply ...,0
6590,"Okay wait, rage part depression?! Can someone ...",1
25769,I debated making Brandin sim I lonely sim alre...,1
24697,For suicide rates lowest I seen 41%. That vary...,1


In [ ]:
export_csv = final_df.to_csv(r'sentiment_depressive.csv')

files.download('sentiment_depressive.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>